# Deep Variational Models for Collaborative Filtering-based Recommender Systems

## Experimental setup

First of all, we include required libreries.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Dropout, Dense, Concatenate, Dot, Lambda
from keras.optimizers import Adam
from keras import backend as K

from sklearn.metrics import precision_score, recall_score, ndcg_score, mean_absolute_error, mean_squared_error, r2_score

Now, we configure the parameters of the experiments. Please, note that each cell contains the configuration for one dataset. Run only the cell of the dataset that you want to evaluate.

In [ ]:
dataset = 'ml1m'
latent_dim = 5
like_threshold = 4
steps_per_epoch = None

deepmf_epochs = 10
ncf_epochs = 10
neumf_epochs = 10
vdeepmf_epochs = 6
vncf_epochs = 9
vneumf_epochs = 5

In [ ]:
dataset = 'ft'
latent_dim = 5
like_threshold = 3
steps_per_epoch = None

deepmf_epochs = 15
ncf_epochs = 8
vdeepmf_epochs = 10
vncf_epochs = 6

In [ ]:
dataset = 'anime'
latent_dim = 7
like_threshold = 8
steps_per_epoch = None

deepmf_epochs = 20
ncf_epochs = 15
vdeepmf_epochs = 9
vncf_epochs = 9

In [ ]:
dataset = 'netflix'
latent_dim = 6
like_threshold = 4
steps_per_epoch = 200000

deepmf_epochs = 5
ncf_epochs = 4
vdeepmf_epochs = 3
vncf_epochs = 3

Dataset loading.

In [ ]:
df = pd.read_csv('./datasets/' + dataset + '.csv', delimiter = ',')

num_users = df.user.max() + 1
num_items = df.item.max() + 1

X = df[['user', 'item']].to_numpy()
y = df[['rating']].to_numpy()

Dataset split into train and test partitions.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = [X_train[:,0], X_train[:,1]]
X_test = [X_test[:,0], X_test[:,1]]

## Neural based Collaborative Filtering models definition

### State of the art models

#### DeepMF

Model architecture:

In [ ]:
user_input = Input(shape=[1])
user_embedding = Embedding(num_users, latent_dim)(user_input)
user_vec = Flatten()(user_embedding)

item_input = Input(shape=[1])
item_embedding = Embedding(num_items, latent_dim)(item_input)
item_vec = Flatten()(item_embedding) 
        
dot = Dot(axes=1)([item_vec, user_vec])
    
DeepMF = Model([user_input, item_input], dot)  

Model fitting using GPU:

In [ ]:
with tf.device('/GPU:1'):
    DeepMF.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
    DeepMF.summary()

    deepmf_report = DeepMF.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=deepmf_epochs, steps_per_epoch=steps_per_epoch, verbose=1)

#### NCF

Model architecture:

In [ ]:
item_input = Input(shape=[1], name='item-input')
item_embedding = Embedding(num_items, latent_dim, name='item-embedding')(item_input)
item_vec = Flatten(name='item-flatten')(item_embedding)

user_input = Input(shape=[1], name='user-input')
user_embedding = Embedding(num_users, latent_dim, name='user-embedding')(user_input)
user_vec = Flatten(name='user-flatten')(user_embedding)

concat = Concatenate(axis=1, name='item-user-concat')([item_vec, user_vec])
fc_1 = Dense(70, name='fc-1', activation='relu')(concat)
fc_1_dropout = Dropout(0.5, name='fc-1-dropout')(fc_1)
fc_2 = Dense(30, name='fc-2', activation='relu')(fc_1_dropout)
fc_2_dropout = Dropout(0.4, name='fc-2-dropout')(fc_2)
fc_3 = Dense(1, name='fc-3', activation='relu')(fc_2_dropout)

NCF = Model([user_input, item_input], fc_3)

Model fitting using GPU:

In [ ]:
with tf.device('/GPU:1'):
    NCF.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
    NCF.summary()
    
    ncf_report = NCF.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=ncf_epochs, steps_per_epoch=steps_per_epoch, verbose=1)

### Proposed models

#### VDeepMF

Model architecture:

In [ ]:
batch_size = 32

def sampling(args):
    z_mean, z_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1)
    return z_mean + K.exp(z_var) * epsilon

user_input = Input(shape=[1])
user_embedding = Embedding(num_users, latent_dim)(user_input)
user_embedding_mean = Dense(latent_dim)(user_embedding)
user_embedding_var = Dense(latent_dim)(user_embedding)
user_embedding_z = Lambda(sampling)([user_embedding_mean, user_embedding_var])
user_vec = Flatten()(user_embedding_z)

item_input = Input(shape=[1])
item_embedding = Embedding(num_items, latent_dim)(item_input)
item_embedding_mean = Dense(latent_dim)(item_embedding)
item_embedding_var = Dense(latent_dim)(item_embedding)
item_embedding_z = Lambda(sampling)([item_embedding_mean, item_embedding_var], latent_dim)
item_vec = Flatten()(item_embedding_z)

dot = Dot(axes=1)([item_vec, user_vec])

VDeepMF = Model([user_input, item_input], dot)   

Model fitting using GPU:

In [ ]:
with tf.device('/GPU:1'):
    VDeepMF.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
    VDeepMF.summary()

    vdeepmf_report = VDeepMF.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=vdeepmf_epochs, batch_size=batch_size, steps_per_epoch=steps_per_epoch, verbose=1)

#### VNCF

Model architecture:

In [ ]:
batch_size = 32

def sampling(args):
    z_mean, z_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1)
    return z_mean + K.exp(z_var) * epsilon

user_input = Input(shape=[1])
user_embedding = Embedding(num_users, latent_dim)(user_input)
user_embedding_mean = Dense(latent_dim)(user_embedding)
user_embedding_var = Dense(latent_dim)(user_embedding)
user_embedding_z = Lambda(sampling)([user_embedding_mean, user_embedding_var])
user_vec = Flatten()(user_embedding_z)

item_input = Input(shape=[1])
item_embedding = Embedding(num_items + 1, latent_dim)(item_input)
item_embedding_mean = Dense(latent_dim)(item_embedding)
item_embedding_var = Dense(latent_dim)(item_embedding)
item_embedding_z = Lambda(sampling)([item_embedding_mean, item_embedding_var], latent_dim)
item_vec = Flatten()(item_embedding_z)

concat = Concatenate(axis=1)([item_vec, user_vec])

fc_1 = Dense(80, name='fc-1', activation='relu')(concat)
fc_1_dropout = Dropout(0.6, name='fc-1-dropout')(fc_1)
fc_2 = Dense(25, name='fc-2', activation='relu')(fc_1_dropout)
fc_2_dropout = Dropout(0.4, name='fc-2-dropout')(fc_2)
fc_3 = Dense(1, name='fc-3', activation='relu')(fc_2_dropout)

VNCF = Model([user_input, item_input], fc_3)

Model fitting using GPU:

In [ ]:
with tf.device('/GPU:1'):
    VNCF.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
    VNCF.summary()

    vncf_report = VNCF.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=vncf_epochs, batch_size=batch_size, steps_per_epoch=steps_per_epoch, verbose=1)

## Experimental results

In [ ]:
methods = ['vdeepmf', 'deepmf', 'vncf', 'ncf']

In [ ]:
preds = pd.DataFrame()

preds['user'] = X_test[0]
preds['item'] = X_test[1]

preds['y_test'] = y_test

In [ ]:
y_true = preds['y_test'].to_numpy()

Store predictions of the baselines:

In [ ]:
preds['deepmf'] = DeepMF.predict(X_test)

In [ ]:
preds['ncf'] = NCF.predict(X_test)

Due to the variational approachs of the proposed methods, the same model can generates different predictions for the same `<user, item>` input. To avoid this, we compute the predictions of the proposed models as the average of 10 repetitions of the same prediction.

In [ ]:
n_repeats = 10

In [ ]:
predictions = None

for i in range(n_repeats):
    if i == 0:
        predictions = VDeepMF.predict(X_test)
    else:
        predictions = np.append(predictions, VDeepMF.predict(X_test), axis=1)
        
preds['vdeepmf'] = np.mean(predictions, axis=1)

In [ ]:
predictions = None

for i in range(n_repeats):
    if i == 0:
        predictions = VNCF.predict(X_test)
    else:
        predictions = np.append(predictions, VNCF.predict(X_test), axis=1)

preds['vncf'] = np.mean(predictions, axis=1)

### Quality of the predictions

In [ ]:
print('MAE:')
for m in methods:
    print('-', m, ':', mean_absolute_error(preds['y_test'], preds[m]))

print('MSD:')
for m in methods:
    print('-', m, ':', mean_squared_error(preds['y_test'], preds[m]))

print('R2:')
for m in methods:
    print('-', m, ':', r2_score(preds['y_test'], preds[m]))

### Quality of the recommendations

In [ ]:
recommendations = {}

for m in methods:
    recommendations[m] = {}

X = preds[['user', 'item']].to_numpy()
for user_id in np.unique(X[:,0]):
    indices = np.where(X[:,0] == user_id)[0]
    
    for m in methods:
        user_pred = np.array([preds[m].to_numpy()[i] for i in indices])
        user_rec = [indices[pos] for pos in np.argsort(user_pred)[::-1]]
        recommendations[m][user_id] = user_rec

In [ ]:
num_recommendations = [2,3,4,5,6,7,8,9,10]

#### Precision, recall and F1

In [ ]:
def recommender_precision_recall(recommendations, y_true, y_pred, N, threshold):
    precision = 0
    recall = 0
    count = 0
    
    for user_id, indices in recommendations.items():

        valid_user = False
        rec_true = np.zeros(len(indices))
        rec_pred = np.zeros(len(indices))
        
        for i, index in enumerate(indices):
            
            if y_true[index] >= threshold:
                rec_true[i] = 1
                valid_user = True
                
            if i < N and y_pred[index] >= threshold:
                rec_pred[i] = 1
                
        if valid_user:
            precision += precision_score(rec_true, rec_pred, zero_division=0)
            recall += recall_score(rec_true, rec_pred)
            count += 1
        
    return precision/count, recall/count

In [ ]:
precision = {}
recall = {}
f1 = {}

for m in methods:
    print(m)
    
    precision[m] = np.zeros(len(num_recommendations))
    recall[m] = np.zeros(len(num_recommendations))
    
    for i, n in enumerate(num_recommendations):
        y_pred = preds[m].to_numpy()
        precision[m][i], recall[m][i] = recommender_precision_recall(recommendations[m], y_true, y_pred, n, like_threshold) 

    f1[m] = 2 * precision[m] * recall[m] / (precision[m] + recall[m])

In [ ]:
for m in methods:
    c = 'blue' if 'deepmf' in m else 'red'
    alpha = 1 if m[0] == 'v' else 0.6
    ls = '-' if m[0] == 'v' else '--'
        
    plt.plot(recall[m], precision[m], c=c, ls=ls, alpha=alpha, label=m)

    if m == 'vdeepmf':
        for i,(r,p) in enumerate(zip(recall[m], precision[m])):
            plt.annotate(num_recommendations[i], (r,p), textcoords="offset points", xytext=(5,5), ha='center')
    
plt.xlabel('Recall', fontsize=15); 
plt.ylabel('Precision', fontsize=15)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.legend(bbox_to_anchor=(0,1.02,1,0.2), fontsize=12, loc="lower left", mode="expand", borderaxespad=0, ncol=len(methods), handlelength=1.2)

plt.grid(True)

ylim_min, ylim_max = plt.ylim()
plt.ylim((ylim_min, ylim_max * 1.02))

plt.savefig('./results/' + dataset + '-precision-recall.png', dpi=300)

plt.show()

In [ ]:
for m in methods:
    c = 'blue' if 'deepmf' in m else 'red'
    alpha = 1 if m[0] == 'v' else 0.6
    ls = '-' if m[0] == 'v' else '--'

    plt.plot(num_recommendations, f1[m], c=c, ls=ls, alpha=alpha, label=m)

plt.xlabel('Number of recommendations', fontsize=15); 
plt.ylabel('F1', fontsize=15)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.legend(bbox_to_anchor=(0,1.02,1,0.2), fontsize=12, loc="lower left", mode="expand", borderaxespad=0, ncol=len(methods), handlelength=1.2)

plt.grid(True)

ylim_min, ylim_max = plt.ylim()
plt.ylim((ylim_min, ylim_max * 1.02))

plt.savefig('./results/' + dataset + '-f1.png', dpi=300)

plt.show()

#### Novelty

In [ ]:
def recommender_novelty(recommendations, y_true, y_pred, num_recommendations):
    novelty = np.zeros(len(num_recommendations))
    count = np.zeros(len(num_recommendations))
    
    bincount = np.bincount(X_test[1])
    
    for user_id, indices in recommendations.items():
        for i, n in enumerate(num_recommendations):
            for index in indices[:n]:
                item_id = X_test[1][index]
                num_item_ratings = bincount[item_id] 
                total_num_ratings = len(X_test[1])
                pi = num_item_ratings / total_num_ratings  
                novelty[i] += -math.log(pi,2)
                count[i] += 1
        
    return novelty/count

In [ ]:
for m in methods:    
    print(m)
    
    y_pred = preds[m].to_numpy()
    novelty = recommender_novelty(recommendations[m], y_true, y_pred, num_recommendations) 

    c = 'blue' if 'deepmf' in m else 'red'
    alpha = 1 if m[0] == 'v' else 0.6
    ls = '-' if m[0] == 'v' else '--'
            
    plt.plot(num_recommendations, novelty, c=c, ls=ls, alpha=alpha, label=m)

plt.xlabel('Number of recommendations', fontsize=15); 
plt.ylabel('Novelty', fontsize=15)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.legend(bbox_to_anchor=(0,1.02,1,0.2), fontsize=12, loc="lower left", mode="expand", borderaxespad=0, ncol=len(methods), handlelength=1.2)

plt.grid(True)

ylim_min, ylim_max = plt.ylim()
plt.ylim((ylim_min, ylim_max * 1.02))

plt.savefig('./results/' + dataset + '-novelty.png', dpi=300)

plt.show()

#### NDCG

In [ ]:
def recommender_ndcg(recommendations, y_true, y_pred, N):
    ndcg = 0
    count = 0
    
    for indices in recommendations.values():     
        if len(indices) > 1:
            user_true = np.array([y_true[i] for i in indices])
            user_pred = np.array([y_pred[i] for i in indices])  
                
            user_true = np.expand_dims(user_true, axis=0)
            user_pred = np.expand_dims(user_pred, axis=0)

            ndcg += ndcg_score(user_true, user_pred, k=N, ignore_ties=False)
            count += 1
    
    return ndcg / count

In [ ]:
for m in methods:
    print(m)
    
    ndcg = np.zeros(len(num_recommendations))
    
    for i, n in enumerate(num_recommendations):
        y_pred = preds[m].to_numpy()
        ndcg[i] = recommender_ndcg(recommendations[m], y_true, y_pred, n) 
        
    c = 'blue' if 'deepmf' in m else 'red'
    alpha = 1 if m[0] == 'v' else 0.6
    ls = '-' if m[0] == 'v' else '--'
 
    plt.plot(num_recommendations, ndcg, c=c, ls=ls, alpha=alpha, label=m)

plt.xlabel('Number of recommendations', fontsize=15); 
plt.ylabel('NDCG', fontsize=15)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.legend(bbox_to_anchor=(0,1.02,1,0.2), fontsize=12, loc="lower left", mode="expand", borderaxespad=0, ncol=len(methods), handlelength=1.2)

plt.grid(True)

plt.savefig('./results/' + dataset + '-ndcg.png', dpi=300)

plt.show()